<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/training_with_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%pip install torch
import torch
if not torch.cuda.is_available():
  %pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q

In [10]:
%pip install wandb numpy transformers datasets transformers[torch] accelerate -U -q



Note: you may need to restart the kernel to use updated packages.


In [9]:
use_wandb = False

if use_wandb:
    import wandb

    wandb.init(project="codeBERTa-on-csharp")

In [11]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
)


model_name = "huggingface/CodeBERTa-small-v1"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("microsoft/LCC_csharp")
tokenizer = AutoTokenizer.from_pretrained(model_name)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [17]:
dataset = dataset.map(
    lambda x: tokenizer(x["context"], truncation=True, padding="max_length"),
    batched=True,
    remove_columns=dataset["train"].column_names,
)

RobertaTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(300))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

block_size = 250


def group_texts(examples):
    # Concatenate all texts.
    # concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    concatenated_examples = examples
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


small_train_dataset = small_train_dataset.map(group_texts)
small_eval_dataset = small_eval_dataset.map(group_texts)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [97]:
small_train_dataset = small_train_dataset.map(
    lambda example: {
        "input_ids": example["input_ids"][0],
        "attention_mask": example["attention_mask"][0],
        "labels": example["labels"][0],
    }
)
small_eval_dataset = small_eval_dataset.map(
    lambda example: {
        "input_ids": example["input_ids"][0],
        "attention_mask": example["attention_mask"][0],
        "labels": example["labels"][0],
    }
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [111]:
small_train_dataset.features

{'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [113]:
import os


model = AutoModelForMaskedLM.from_pretrained(model_name)


if wandb:

    os.environ["WANDB_PROJECT"] = "codeBERTa-on-csharp"

    os.environ["WANDB_LOG_MODEL"] = "true"

    os.environ["WANDB_WATCH"] = "false"



# pass "wandb" to the 'report_to' parameter to turn on wandb logging



training_args = TrainingArguments(
    output_dir="models",

    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,

    weight_decay=0.01,
    # push_to_hub=True,

)
if wandb:
    training_args.report_to = ["wandb"]

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [114]:
trainer.train()

  0%|          | 0/37500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

if wandb:
    wandb.finish()

In [ ]:
from transfromers import pipeline

text = """
//
// Author:
//   Jb Evain (jbevain@gmail.com)
//
// Copyright (c) 2008 - 2015 Jb Evain
// Copyright (c) 2008 - 2011 Novell, Inc.
//
// Licensed under the MIT/X11 license.
//

namespace Mono.Cecil.Cil {

	public sealed class VariableDefinition : VariableReference {

		public bool IsPinned {
			get { return variable_type.IsPinned; }
		}

		public VariableDefinition (TypeReference variableType)
			: base (variableType)
		{
		}

		public override VariableDefinition Resolve ()
		{
			return this;
		}
	}
}"""

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)
pipe(text)